In [2]:
from examples import acy_util
from qml.tools.portfolio_optimize.cvx_optimize import efficient_frontier_non_smooth
import jax
import xarray as xr

xr.set_options(display_style="html")

This notebook studies the minimization of CVaR.

Let's start out by getting some parameters inspired by Tables 9-11 in the Alexander/Coleman/Li paper.

In [3]:
params = acy_util.acy_params_with_risk_free_rate()
params

<xarray.Dataset>
Dimensions:      (asset: 5, time_step: 1)
Dimensions without coordinates: asset, time_step
Data variables:
    mu           (asset) float64 0.1091 0.0619 0.0279 0.0649 0.01
    sigma        (asset) float64 0.5376 0.3406 0.1483 0.2811 0.0
    C_matrix     (asset, asset) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0
    start_value  (asset) float64 100.0 50.0 30.0 100.0 1.0
    time_steps   (time_step) float64 0.25

Let's use these parameters to generate some asset price paths.


In [4]:
from qml.tools.monte_carlo.brownian_motion import xr_vector_geometric_brownian_motion

seed = 42
key = jax.random.PRNGKey(seed)  # This initializes a key for the jax pseudo-random-number generator
asset_paths = xr_vector_geometric_brownian_motion(num_paths=10000, params=params, key=key)[:, -1, :]
asset_paths.shape

(10000, 5)

The two-dimensional array *asset_paths* records the price of each of four underlying assets at the terminal time in each simulation.

Before turning to CVaR minimization with call options, lets look at CVaR minimization with the four underlying assets.  We will make use of CVX to solve the optimization problem for a range of points on the efficient frontier.

In [ ]:
efficient_frontier = efficient_frontier_non_smooth(
    confidence_level=0.95,  # i.e. we want to use CVAR 95
    instrument_price=params.start_value,
    instrument_payoff=asset_paths,
    required_returns=[0.08, 0.05, 0.02, 0.01],
)
efficient_frontier

Let's visualize this efficient frontier.

In [ ]:
from qml.tools.portfolo_plot.cvar_min_plot import plot_mean_cvar_frontier, add_risk_return_percentage_to_ds

efficient_frontier = add_risk_return_percentage_to_ds(efficient_frontier)
figure = plot_mean_cvar_frontier(ds=efficient_frontier)
figure

We can also visualize how asset holdings change as we move along the frontier.

In [ ]:
from qml.tools.portfolo_plot.cvar_min_plot import plot_asset_holdings

holding_figure = plot_asset_holdings(efficient_frontier)
holding_figure